<img src='./img/LogoWekeo_Copernicus_RGB_0.png' align='right' width='20%'></img>

# Tutorial on basic land applications (data download)

In this tutorial we will use the WEkEO Jupyterhub to access and analyse data from the Copernicus Sentinel-2 and products from the [Copernicus Land Monitoring Service (CLMS)](https://land.copernicus.eu/). 
A region in northern Corsica has been selected as it contains representative landscape features and process elements which can be used to demonstrate the capabilities and strengths of Copernicus space component and services.
The tutorial comprises the following steps:
1. [Search and download data](#search_download): We will select and download a Sentinel-2 scene and the CLMS CORINE Land Cover (CLC) data from their original archive locations via WEkEO using the Harmonised Data Access (HAD) API.
2.	Read and view Sentinel-2 data: Once downloaded, we will read and view the Sentinel-2 data in geographic coordinates as true colour image.
3.	Process and view Sentinel-2 data as a vegetation index: We will see how the vegetation density and health can be assessed from optical EO data to support crop and landscape management practices.
4.	Read and view the CLC data: Display the thematic CLC data with the correct legend.
5.	CLC2018 burnt area in the Sentinel-2 NDVI data: The two products give different results, but they can be combined to provide more information.

<img src='./img/Intro_banner.jpg' align='center' width='100%'></img>

## <a id='search_download'></a>1. Search and download data

Before we begin we must prepare our environment. This includes importing the various python libraries that we will need.

### Load required libraries

In [ ]:
import os
import sys
import json
import time
import base64
from IPython.core.display import HTML

import requests
import warnings
warnings.filterwarnings('ignore')
import zipfile

### Install the WEkEO HDA client

The WEkEO HDA client is a python based library. It provides support for both Python 2.7.x and Python 3.

In order to install the WEkEO HDA client via the package management system pip, you have to running on Unix/Linux the command shown below.

In [ ]:
pip install -U hda

Please verify the following requirements are installed before skipping to the next step:
   - Python 3
   - requests
   - tqdm

#### Load WEkEO HDA client

The hda client provides a fully compliant Python 3 client that can be used to search and download products using the Harmonized Data Access WEkEO API.
HDA is RESTful interface allowing users to search and download WEkEO datasets.
Documentation about its usage can be found at https://www.wekeo.eu/.

In [ ]:
from hda import Client

<hr>

### Search for the dataset ID from the WEkEO landing platform

The [WEkEO land platform](https://wekeo.eu/) offers access to a huge amount of data. Under [WEkEO DATA](https://wekeo.eu/data?view=catalogue), clicking the + to add a layer, opens a catalogue search. Here you can use free text, or you can use the filter options on the left to refine your search and look up data by satellite platform, sensor, Copernicus service, area (region of interest), general time period (past or future), as well as through a variety of flags.

<img src='./img/WEkEO_data_01.jpg' align='middle' width='50%'></img>

You can click on the datasets you are interested in and you will be guided to a range of details including the dataset temporal and spatial extent, collection ID, and metadata.

When searching for the products of Sentinel-2, click under Platform in the Filters on the left-hand side of the catalogue panel. Two datasets are available, but we will use “SENTINEL-2 Level-1C”. Once you have found it, select 'Details' to read the dataset description.
The dataset description provides the following information:
* Abstract, containing a general description of the dataset,
* Classification, including the Dataset ID
* Resources, such as a link to the Product Data Format Specification guide, and JSON metadata
* Contacts, where you can find further information about the data source from its provider.
* Raw metadata, details of the dataset in XML format.

<img src='./img/WEkEO_data_02.jpg' align='centre' width='50%'></img>

You need the information from here to request data from the Harmonised Data Access API. 

This process is explained in a previous training session which can be found at the [WEkEO YouTube Channel](https://www.youtube.com/channel/UCvS3VvKmMKs1M2ZkmQPyRlw). The YouTube channel also contains lots of other useful training and support information, such as how to [clone the GitHub to refresh the training materials](https://www.youtube.com/watch?v=pmCkvXcnZxY&list=PLAT-b7DuvMgogqJa5_ii5GteOYmXCce24&index=2).

For this session the details of the required datasets have already been prepared as JSON files which are used below.


In [ ]:
dataset_id_S2 = "EO:EO:ESA:DAT:SENTINEL-2:MSI"
dataset_id_corine = "EO:CLMS:DAT:CORINE"

filename_json_S2 = os.path.join(os.getcwd(),'S2_request.json') 
filename_json_corine = os.path.join(os.getcwd(), 'corine_corsica.json')

### Configure the WEkEO API Authentication

In order to interact with WEkEO's Harmonised Data Access API, each user first makes sure the file "$HOME/.hdarc" exists with the URL to the API end point and your user and password.

For example, to search for the file .hdarc in the $HOME diretory, the user would open a terminale and run the following command:

Then he could copy the code below in the file "$HOME/.hdarc" (in your Unix/Linux environment) and adapt the following template with the credentials of your WEkEO account:

If he doesn't have a WEkEO account, please self register at the WEkEO registration page https://my.wekeo.eu/web/guest/user-registration.

### Load data descriptor file and request data
The Harmonised Data Access API can read your data request from a JSON file. In this JSON-based file, you can describe the dataset you are interested in downloading. The file is in principle a dictionary. Following keys can be defined:

- datasetID: the dataset's collection ID
- stringChoiceValues: type of dataset, e.g. 'Non Time Critical'
- dataRangeSelectValues: time period you would like to retrieve data
- boundingBoxValues: optional to define a subset of a global field

You can also get a specific example of a JSON file for a particular query from the WEkEO DATA portal.

### Displaying a JSON query from a request made to the Harmonised Data Access API through the data portal

You can load the JSON file with json.load(). Alternatively, you can copy paste the dictionary describing your data into a cell as shown in the YouTube video.

For this training session JSON file has already been prepared to select the correct Sentinel-2 scene and CLC data to correspond with later tasks. The details were loaded earlier in the JN. The following cell reads that JSON file and displays the contents.

In [ ]:
try:
    with open(filename_json_S2, 'r') as f:
        data_S2 = json.load(f)
    print('Your JSON file:')
    print(data_S2)
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

In [ ]:
try:
    with open(filename_json_corine, 'r') as f:
        data_corine = json.load(f)
    print('Your JSON file:')
    print(data_corine)
except:
    print('Your JSON file is not in the correct format, or is not found, please check it!')

### Download requested data

You can use directly the client to download data as in following example. 

In [ ]:
c = Client(debug=True)

matches = c.search(data_S2)
print(matches)
matches.download()


c = Client(debug=True)

matches = c.search(data_corine)
print(matches)
matches.download()

### Decompressing Sentinel 2 and Corine Land Cover data

In [ ]:
#rename S2 file
download_dir_path = os.getcwd()
for item in os.listdir(download_dir_path): 
    if item.endswith('SAFE'):
        os.rename(os.path.join(download_dir_path, item), os.path.join(download_dir_path, item +".zip"))

In case you have permission issues when renaming the .SAFE file with the code above, please manually do that by right-clicking the file, then rename, then add .zip to the end of the filename.

For example:
- From: data/from_wekeo/S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE   
- To: data/from_wekeo/S2A_MSIL2A_20170802T101031_N0205_R022_T32TNN_20170802T101051.SAFE.zip

In [ ]:
extension = ".zip"
for item in os.listdir(download_dir_path): # loop through items in dir
    print(item)
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.join(download_dir_path, item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(download_dir_path) # extract file to dir
        zip_ref.close() # close file

<hr>

<p><img src='./img/all_partners_wekeo_2.png' align='left' alt='Logo EU Copernicus' width='100%'></img></p>